In [183]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input

In [13]:
df = pd.read_csv("VAE_Gaussian_augmented_dataset.csv")
#test_df = pd.read_csv("test.csv")
# print(test_df.shape)
# print(test_df.isnull().values.any())
# test_df.head()

In [15]:
#df = pd.read_csv("VAE_Gaussian_augmented_dataset.csv")
print(df.shape)
print(df.isnull().values.any())
df.head()

(9288, 53)
False


,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,max_TCP_qy,mean_TCP_qz,std_TCP_qz,min_TCP_qz,max_TCP_qz,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,class
0,0.091094,0.551535,-2.716940,5.026344,-0.389788,1.482441,-14.920257,2.075711,-0.551167,1.995361,...,0.114844,-0.067169,0.210216,-0.397239,0.328366,0.007217,0.715606,-0.983862,0.992033,4
1,0.042475,0.558085,-4.307300,8.625980,-0.271580,1.426492,-16.193495,2.097393,-0.349012,1.884769,...,0.080914,-0.065886,0.201699,-0.372874,0.344875,-0.050501,0.713410,-0.989799,0.990807,4
2,0.061610,0.624525,-4.413770,10.329532,-0.310435,2.127228,-28.516469,2.023712,-0.410629,2.892710,...,0.081576,-0.076213,0.194011,-0.370489,0.328109,-0.129140,0.704608,-0.992530,0.987084,4
3,0.108036,0.733039,-6.900487,8.984035,-0.593093,2.149323,-21.772929,2.335922,-0.784609,2.876551,...,0.006726,-0.084439,0.195364,-0.416887,0.328214,-0.166065,0.686092,-0.996298,0.921216,4
4,0.048892,0.503397,-3.754972,4.944276,-0.260897,1.332355,-15.793961,1.640494,-0.354656,1.820410,...,0.091534,-0.070010,0.221369,-0.406463,0.413101,-0.041900,0.711247,-0.994797,0.986257,4


In [19]:
X = df.drop(columns=["class"])  # Features: all columns except the label
y = df["class"]  # Class labels

# Encode the labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding for categorical cross-entropy
y_onehot = to_categorical(y_encoded)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the data for 1D CNN (samples, timesteps, features)
# Timesteps = 1 since we are treating each task as one sequence
X_reshaped = X_scaled[..., np.newaxis]  # Adding 1 channel for CNN input

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_onehot, test_size=0.2, random_state=42
)

In [21]:
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),  # Explicit input layer
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_onehot.shape[1], activation='softmax')  # Output layer with softmax for multi-class
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 50, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 25, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 25, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 23, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        90,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,092 (379.27 KB)

 Trainable params: 97,092 (379.27 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

# Step 5: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6071 - loss: 0.9601 - val_accuracy: 0.8520 - val_loss: 0.4097
Epoch 2/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8337 - loss: 0.4795 - val_accuracy: 0.8794 - val_loss: 0.3281
Epoch 3/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8713 - loss: 0.3735 - val_accuracy: 0.8977 - val_loss: 0.2797
Epoch 4/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8775 - loss: 0.3467 - val_accuracy: 0.9026 - val_loss: 0.2478
Epoch 5/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8920 - loss: 0.2795 - val_accuracy: 0.9160 - val_loss: 0.2256
Epoch 6/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9006 - loss: 0.2759 - val_accuracy: 0.9257 - val_loss: 0.1993
Epoch 7/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9050 - loss: 0.2544 - val_accuracy: 0.9279 - val_loss: 0.1992
Epoch 8/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9109 - loss: 0.2385 - val_accuracy: 0.

In [315]:
df = pd.read_csv("VAE_Gaussian_augmented_train.csv")
test_df = pd.read_csv("test.csv")
print(df.shape)
print(test_df.shape)
print(test_df.isnull().values.any())
test_df.head()

(5030, 53)
(129, 53)
False


,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,max_TCP_qy,mean_TCP_qz,std_TCP_qz,min_TCP_qz,max_TCP_qz,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,class
0,0.011254,0.442999,-1.920936,2.064664,-0.312191,1.293969,-9.808524,1.843434,-0.411665,1.499849,...,0.047622,-0.085202,0.245813,-0.479852,0.368453,-0.113048,0.671682,-0.979342,0.979416,4
1,-0.028499,0.619903,-3.646468,4.933733,-0.399350,1.676785,-13.139836,2.169235,-0.526530,1.987745,...,0.184625,0.026310,0.243701,-0.526289,0.518053,0.144180,0.707204,-0.981018,0.981118,4
2,0.027236,1.018709,-3.262223,3.485886,0.002927,0.744631,-2.493998,3.280509,0.224399,0.712818,...,0.268659,0.006792,0.174839,-0.486699,0.333708,-0.119803,0.700712,-0.994174,0.995862,3
3,0.023499,2.058170,-8.664618,8.899791,0.280381,1.923219,-8.294284,8.722299,0.450387,1.639716,...,0.100171,-0.063907,0.142825,-0.437442,0.327301,0.054084,0.737196,-0.994185,0.999513,2
4,0.051871,0.493422,-2.070790,2.273246,-0.156795,0.486445,-5.142684,1.491303,-0.199192,0.494531,...,0.061934,-0.058214,0.204305,-0.435687,0.335504,-0.071536,0.680499,-0.991251,0.989002,1


In [317]:
X = df.drop(columns=["class"])  # Features: all columns except the label
y = df["class"]  # Class labels

# Encode the labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding for categorical cross-entropy
y_onehot = to_categorical(y_encoded)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the data for 1D CNN (samples, timesteps, features)
# Timesteps = 1 since we are treating each task as one sequence
X_reshaped = X_scaled[..., np.newaxis]  # Adding 1 channel for CNN input

# X_train, X_test, y_train, y_test = train_test_split(
#     X_reshaped, y_onehot, test_size=0.2, random_state=42
# )

In [145]:
filter_sizes = [16, 32, 64]
filter2_sizes = [16, 32, 64, 128]
denses = [16, 32, 64, 128]
kernel_sizes = [3]
pool_sizes = [2,3]
batch_sizes = [32]
print(f'Total models tested: {3*4*4*1*2*1}')
ct = 0
acc_list = []
for f1 in filter_sizes:
    for f2 in filter2_sizes:
        for d in denses:
            for ks in kernel_sizes:
                for ps in pool_sizes:
                    for bs in batch_sizes:
                        ct+=1
                        model = Sequential([
                            Input(shape=(X_train.shape[1], 1)),  # Explicit input layer
                            Conv1D(filters=f1, kernel_size=ks, activation='relu'),
                            MaxPooling1D(pool_size=ps),
                            Dropout(0.2),
                            
                            Conv1D(filters=f2, kernel_size=ks, activation='relu'),
                            MaxPooling1D(pool_size=ps),
                            Dropout(0.2),
                            
                            Flatten(),
                            Dense(d, activation='relu'),
                            Dropout(0.3),
                            Dense(y_onehot.shape[1], activation='softmax')  # Output layer with softmax for multi-class
                        ])
                        
                        model.compile(optimizer='adam',
                                      loss='categorical_crossentropy',
                                      metrics=['accuracy'])
                        
                        history = model.fit(
                            X_reshaped, y_onehot,
                            epochs=25,
                            batch_size=bs,
                            validation_data=(X_train, y_train),
                            verbose=0
                        )

                        testset_loss, testset_accuracy = model.evaluate(X_test_reshaped, y_test_onehot, verbose=1)
                        acc_list.append(([f1, f2, d, ks, ps, bs], testset_accuracy))
                        if ct%100 ==0:
                            print(f"the {ct}th model training completed")
print('Done!!')
print(f'highest test accuracy achieved: {max([i[1] for i in acc_list])}')

Total models tested: 96
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6038 - loss: 1.5573
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5832 - loss: 1.0280 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5264 - loss: 1.5700 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5870 - loss: 1.0823 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6700 - loss: 1.2811 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6125 - loss: 1.0210
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5805 - loss: 1.7235
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5745 - loss: 1.0712 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6346 - loss: 1.0075 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5714 - loss: 1.1933 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6493 - loss: 1.1321 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6017 - loss: 1.0091 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5727 - loss: 1.7536 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accu

In [153]:
sorted(acc_list, key=lambda x: x[1], reverse = True)[:10]

[([16, 16, 64, 3, 2, 32], 0.7054263353347778),
 ([32, 16, 32, 3, 2, 32], 0.6899224519729614),
 ([64, 32, 128, 3, 2, 32], 0.6899224519729614),
 ([16, 64, 128, 3, 3, 32], 0.6744186282157898),
 ([32, 128, 32, 3, 2, 32], 0.6744186282157898),
 ([16, 32, 32, 3, 2, 32], 0.6666666865348816),
 ([16, 64, 32, 3, 2, 32], 0.6666666865348816),
 ([32, 16, 64, 3, 2, 32], 0.6666666865348816),
 ([16, 32, 16, 3, 2, 32], 0.6589147448539734),
 ([16, 128, 64, 3, 2, 32], 0.6589147448539734)]

In [ ]:
X_testset = test_df.drop(columns=["class"])  # Features: all columns except the label
y_testset = test_df["class"]  # Class labels

# Encode the labels to integers
#label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_testset)

# Convert labels to one-hot encoding for categorical cross-entropy
y_test_onehot = to_categorical(y_test_encoded)

# Normalize the features
#scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_testset)

# Reshape the data for 1D CNN (samples, timesteps, features)
# Timesteps = 1 since we are treating each task as one sequence
X_test_reshaped = X_test_scaled[..., np.newaxis]  # Adding 1 channel for CNN input

# Step 5: Evaluate the model
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test Accuracy: {test_accuracy:.4f}")

In [323]:
#kernel_size = 
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
model = Sequential([
    Input(shape=(X_reshaped.shape[1], 1)),  # Explicit input layer
    Conv1D(filters=16, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_onehot.shape[1], activation='softmax'),  # Output layer with softmax for multi-class
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'F1Score']
             )
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_reshaped, y_onehot,
    epochs=30,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_onehot),
    verbose=1#,
    #callbacks=[early_stopping]
)

Epoch 1/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - F1Score: 0.5520 - accuracy: 0.5519 - loss: 1.0669 - val_F1Score: 0.5625 - val_accuracy: 0.5736 - val_loss: 1.1215
Epoch 2/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 0.7713 - accuracy: 0.7679 - loss: 0.6865 - val_F1Score: 0.5903 - val_accuracy: 0.6202 - val_loss: 1.0153
Epoch 3/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 0.7892 - accuracy: 0.7859 - loss: 0.5786 - val_F1Score: 0.6455 - val_accuracy: 0.6667 - val_loss: 0.9077
Epoch 4/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 0.8134 - accuracy: 0.8122 - loss: 0.5871 - val_F1Score: 0.6672 - val_accuracy: 0.6822 - val_loss: 0.9019
Epoch 5/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 0.8398 - accuracy: 0.8364 - loss: 0.4538 - val_F1Score: 0.6384 - val_accuracy: 0.6512 - val_loss: 0.9352
Epoch 6/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - F1Score: 0.8430 - accuracy: 0.8413 - loss: 0.4391 - val_F1Score: 0.6706 - val_accuracy: 0.6822 - val_loss:

In [319]:
X_testset = test_df.drop(columns=["class"])  # Features: all columns except the label
y_testset = test_df["class"]  # Class labels

# Encode the labels to integers
#label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_testset)

# Convert labels to one-hot encoding for categorical cross-entropy
y_test_onehot = to_categorical(y_test_encoded)

# Normalize the features
#scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_testset)

# Reshape the data for 1D CNN (samples, timesteps, features)
# Timesteps = 1 since we are treating each task as one sequence
X_test_reshaped = X_test_scaled[..., np.newaxis]  # Adding 1 channel for CNN input

# Step 5: Evaluate the model
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
testset_loss, testset_accuracy, test_f1 = model.evaluate(X_test_reshaped, y_test_onehot, verbose=1)
print(f"Test Loss: {testset_loss:.4f}")
print(f"Test Accuracy: {testset_accuracy:.4f}")
print(f"Test Accuracy: {np.mean(test_f1)}")

In [311]:
#cross validation
test_set_accs = []
test_set_f1s = []
for i in range(10):
    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=0
    )

    # Step 5: Evaluate the model
    #test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    #print(f"Train Split Test Loss: {test_loss:.4f}")
    #print(f"Train Split Test Accuracy: {test_accuracy:.4f}")

    testset_loss, testset_accuracy, test_f1 = model.evaluate(X_test_reshaped, y_test_onehot, verbose=1)
    #print(f"Test Loss: {testset_loss:.4f}")
    #print(f"Test Accuracy: {testset_accuracy:.4f}")
    test_set_accs.append(testset_accuracy)
    test_set_f1s.append(np.mean(test_f1))

print(f"Mean Test_set Accuracy across 10 fold cross validation: {np.mean(test_set_accs):.4f}, f1 is {np.mean(test_set_f1s)}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - F1Score: 0.7041 - accuracy: 0.7069 - loss: 1.1311 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.6846 - accuracy: 0.6818 - loss: 1.3261
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.6838 - accuracy: 0.6818 - loss: 1.5765 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.7063 - accuracy: 0.7005 - loss: 1.6259 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - F1Score: 0.6969 - accuracy: 0.6918 - loss: 1.5327 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.6811 - accuracy: 0.6693 - loss: 1.9985 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.7038 - accuracy: 0.6943 - loss: 1.9488 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - F1Score: 0.7010 - accuracy: 0.6935 - loss: 1.8720 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - F1Score: 0.6973 - accuracy: 0.6896 - loss: 2.0165 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - F1Score: 0.6827 - accuracy: 0.6836 - loss: 2.1632 
Mean Test_set Accuracy across 10 fold cross validation: 0.6922, f1 is 0.6927462816238403


In [ ]:
#Model 2

In [ ]:
df = pd.read_csv("VAE_UnconGaussian_train_downsample.csv")
test_df = pd.read_csv("test_downsample.csv")
print(test_df.shape)
print(test_df.isnull().values.any())
test_df.head()